In [31]:
import gc, cv2, os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, BatchNormalization, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

In [7]:
traindf = pd.read_csv('train.csv', dtype=str)
testdf = pd.read_csv('test.csv', dtype=str)

In [8]:
traindf.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [9]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=10,
                             width_shift_range=0.1, 
                             height_shift_range=0.1)

train_generator = datagen.flow_from_dataframe(
dataframe=traindf,
directory="./Train Images/",
x_col="Image",
y_col="Class",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224, 224))

Found 5983 validated image filenames belonging to 4 classes.


In [10]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./Test Images/",
x_col="Image",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224, 224))

Found 3219 validated image filenames.


In [1]:
resnet_model = applications.ResNet50(weights = "imagenet", include_top = False, input_shape = (224, 224, 3))

for layer in resnet_model.layers[:-6]:
    layer.trainable = False
    
df = pd.DataFrame(([layer.name, layer.trainable] for layer in model.layers), columns=['layer', 'trainable'])
df.style.applymap(lambda trainable: f'background-color: {"yellow" if trainable else "red"}', subset=['trainable'])

NameError: name 'applications' is not defined

In [42]:
input = Input(shape=(224, 224, 3),name = 'image_input')
x = resnet_model(input)

#x = BatchNormalization()(x)
#x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
#x = Dropout(0.2)(x)

#x = BatchNormalization()(x)
#x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
#x = Dropout(0.2)(x)

x = GlobalAveragePooling2D()(x)

#x = Flatten()(x)
#x = Dense(4096, activation='relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.3)(x)

x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(4, activation='softmax')(x)

model = Model(input=input, output=x)

model.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_8 ( (None, 2048)              0         
_________________________________________________________________
dense_62 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_57 (Batc (None, 1024)              4096      
_________________________________________________________________
dropout_57 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_63 (Dense)             (None, 512)               524

c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("de...)`


In [43]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [45]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=1,
                    verbose=1)

Epoch 1/1
 72/186 [==========>...................] - ETA: 37:45 - loss: 1.3779 - accuracy: 0.5516

KeyboardInterrupt: 

In [ ]:
test_generator.reset()
pred = model.predict_generator(test_generator,
                               steps=STEP_SIZE_TEST,
                               verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred, axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames

results=pd.DataFrame({"Image":filenames,
                      "Class":predictions})

results.to_csv("results.csv", index = False)